<a href="https://colab.research.google.com/github/matthewreed2000/cse380-notebooks/blob/master/CSE380_Teach10_Pairing_Off.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [1]:
import numpy as np

In [171]:
one_to_one = np.array([
  [1, 0, 0],
  [0, 1, 0],
  [0, 0, 1]
])

anybody = np.array([
  [1, 1, 1],
  [1, 1, 1],
  [1, 1, 1]
])

nobody = np.array([
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]
])

cannot = np.array([
  [0, 1, 0],
  [1, 0, 1],
  [0, 1, 0]
])

can = np.array([
  [1, 1, 0],
  [1, 1, 1],
  [0, 1, 0]
])

should_work = np.ones((500,500))

should_not_work = np.array([
  [1, 0, 1],
  [0, 0, 0],
  [1, 0, 1]
])

## Explore

### Initial Implementation

In [159]:
def is_pairable(matrix, rep=None, verbose=False):
  assert len(matrix) == len(matrix[0])
  n = len(matrix)

  if rep is None:
    start_rep = n ** 5
  else:
    start_rep = rep

  return is_pairable_rec(matrix, n, start_rep, verbose)

def is_pairable_rec(matrix, n, rep, verbose=False):
  if rep == 0:
    row_sums = [*map(lambda r:sum(r), matrix)]
    col_sums = [*map(lambda c:sum(c), matrix.transpose())]

    if verbose:
      print(rep)
      print(matrix)

    return (max(row_sums + col_sums) <= (1 + 1/n)) and (min(row_sums + col_sums) >= (1 - 1/n))
  else:
    row_sums = np.array([*map(lambda r:sum(r), matrix)])
    if 0 in row_sums: # If the person cannot work with anyone
      if verbose:
        print(rep)
        print(matrix)
      return False
    row_mat = (matrix.transpose() * (1 / row_sums)).transpose()
    col_sums = np.array([*map(lambda c:sum(c), row_mat.transpose())])
    if 0 in col_sums: # If the person cannot work with anyone
      if verbose:
        print(rep)
        print(matrix)
      return False
    col_mat = row_mat * (1 / col_sums)
    return is_pairable_rec(col_mat, n, rep - 1, verbose)

In [160]:
is_pairable(one_to_one, verbose=True)

0
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


True

In [161]:
is_pairable(anybody, verbose=True)

0
[[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]]


True

In [162]:
is_pairable(nobody, verbose=True)

243
[[0 0 0]
 [0 0 0]
 [0 0 0]]


False

In [163]:
is_pairable(cannot, verbose=True)

0
[[0.  0.5 0. ]
 [1.  0.  1. ]
 [0.  0.5 0. ]]


False

In [164]:
is_pairable(can, verbose=True)

0
[[9.95948814e-01 4.04294655e-03 0.00000000e+00]
 [4.05118614e-03 1.64453522e-05 1.00000000e+00]
 [0.00000000e+00 9.95940608e-01 0.00000000e+00]]


True

In [165]:
is_pairable(can, rep=2945, verbose=True)

0
[[9.99661026e-01 3.38916190e-04 0.00000000e+00]
 [3.38973661e-04 1.14922618e-07 1.00000000e+00]
 [0.00000000e+00 9.99660969e-01 0.00000000e+00]]


True

In [166]:
is_pairable(cannot, rep=2945, verbose=True)

0
[[0.e+00 5.e-01 0.e+00]
 [1.e+00 0.e+00 1.e+00]
 [0.e+00 5.e-01 0.e+00]]


False

In [169]:
is_pairable(should_work, rep=5, verbose=True)

0
[[0.002 0.002 0.002 ... 0.002 0.002 0.002]
 [0.002 0.002 0.002 ... 0.002 0.002 0.002]
 [0.002 0.002 0.002 ... 0.002 0.002 0.002]
 ...
 [0.002 0.002 0.002 ... 0.002 0.002 0.002]
 [0.002 0.002 0.002 ... 0.002 0.002 0.002]
 [0.002 0.002 0.002 ... 0.002 0.002 0.002]]


True

In [173]:
is_pairable(should_not_work, verbose=True)

243
[[1 0 1]
 [0 0 0]
 [1 0 1]]


False

### Using Generators for Debugging

In [110]:
def is_pairable_gen(matrix):
  assert len(matrix) == len(matrix[0])
  n = len(matrix)
  return is_pairable_gen_rec(matrix, n, n ** 5)

def is_pairable_gen_rec(matrix, n, rep):
  if rep == 0:
    pass
  else:
    row_sums = np.array([*map(lambda r:sum(r), matrix)])
    row_mat = (matrix.transpose() * (1 / row_sums)).transpose()
    yield row_mat

    col_sums = np.array([*map(lambda c:sum(c), row_mat.transpose())])
    col_mat = row_mat * (1 / col_sums)
    yield col_mat
    yield from is_pairable_gen_rec(col_mat, n, rep - 1)

In [111]:
gen_can = is_pairable_gen(can)

In [112]:
next(gen_can)

array([[0.5       , 0.5       , 0.        ],
       [0.33333333, 0.33333333, 0.33333333],
       [0.        , 1.        , 0.        ]])

In [113]:
[*is_pairable_gen(can)]

[array([[0.5       , 0.5       , 0.        ],
        [0.33333333, 0.33333333, 0.33333333],
        [0.        , 1.        , 0.        ]]),
 array([[0.6       , 0.27272727, 0.        ],
        [0.4       , 0.18181818, 1.        ],
        [0.        , 0.54545455, 0.        ]]),
 array([[0.6875    , 0.3125    , 0.        ],
        [0.25287356, 0.11494253, 0.63218391],
        [0.        , 1.        , 0.        ]]),
 array([[0.73109244, 0.218923  , 0.        ],
        [0.26890756, 0.0805234 , 1.        ],
        [0.        , 0.7005536 , 0.        ]]),
 array([[0.76955848, 0.23044152, 0.        ],
        [0.19927478, 0.05967212, 0.7410531 ],
        [0.        , 1.        , 0.        ]]),
 array([[0.79431468, 0.1786211 , 0.        ],
        [0.20568532, 0.04625338, 1.        ],
        [0.        , 0.77512552, 0.        ]]),
 array([[0.81641018, 0.18358982, 0.        ],
        [0.16429345, 0.0369454 , 0.79876115],
        [0.        , 1.        , 0.        ]]),
 array([[0.83247391,